# Filter Sandbox
This notebook contains functions for testing out DecimationScheme objects on realistic raw I&Q samples, for the purposes of exploring filter performance.

In [ ]:
import sys
import os
import numpy as np
import numpy.fft as nft
sys.path.append(os.environ['BOREALISPATH'])
from src.utils.signals import DSP
try:
    import ipympl    # If you have this installed, you can interact with the plots (zoom, drag, etc.)
    %matplotlib widget
except ImportError:
    pass
    %matplotlib inline

import matplotlib.pyplot as plt

## Defining visualization functions
Next, we define some functions to aid in evaluating the performance of filtering strategies. **Change the figure size below to something suitable for your screen.**

In [ ]:
figsize = (10, 6)

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

def plot_stage_phase_response_on_ax(fig, ax, stage, rxrate, **kwargs):
    if len(stage.filter_taps) < 16:
        taps = np.zeros(16)
        taps[:len(stage.filter_taps)] = stage.filter_taps
        fft_size = 16
    else:
        taps = stage.filter_taps
        
    filter_fft = nft.fft(taps)
    ax.plot(nft.fftshift(nft.fftfreq(filter_fft.size, d=1/rxrate)),
            np.unwrap(nft.fftshift(np.angle(filter_fft))), 
            **kwargs)
    ax.tick_params('y', colors='orange')
    ax.set(xlabel='Frequency (Hz)', ylabel='Phase (rad)')


def plot_stage_freq_response_on_ax(fig, ax, stage, rxrate, **kwargs):
    fft_size = len(stage.filter_taps)

    if fft_size < 32:
        taps = np.zeros(32)
        taps[:fft_size] = stage.filter_taps
        fft_size = 32
    else:
        taps = stage.filter_taps

    ax.plot(nft.fftshift(nft.fftfreq(fft_size, d=1/rxrate)),
            # nft.fftshift(np.abs(nft.fft(stage.filter_taps, n=fft_size))),
            20 * nft.fftshift(np.log10(np.abs(nft.fft(taps, n=fft_size)))), 
            **kwargs)
    ax.set_ylabel('Magnitude (dB)')
    # ax.tick_params('y', colors='blue')
    ax.set(xlabel='Frequency (Hz)', ylabel='Magnitude (dB)')


def plot_freq_response(data, freqs, title, filename, save: bool = False, phase: bool = True):
    fig, ax = plt.subplots(figsize=figsize)

    ax.plot(nft.fftshift(freqs), 20 * nft.fftshift(np.log10(np.abs(data))), color='blue')
    ax.set_ylabel('Magnitude (dB)')
    ax.tick_params('y', colors='blue')
    ax.set(title=title, xlabel='Frequency (Hz)', ylabel='Magnitude (dB)')

    if phase:
        ax2 = ax.twinx()
        ax2.plot(nft.fftshift(freqs), nft.fftshift(np.unwrap(np.angle(data))), color='orange')
        ax2.tick_params('y', colors='orange')
        ax2.set(ylabel='Phase (rad)')

    plt.show()
    return fig, ax


def plot_filter_stage(input_data, output_data, filter_taps, input_rate, output_rate, title, center_freq=0.0, freq_lim=None):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize)
    fig.suptitle(title)

    fig.tight_layout(pad=3)

    fft_size = input_data.size + filter_taps.size - 1

    # Input rate plot
    ax1.plot(nft.fftshift(nft.fftfreq(input_data.size, d=1/input_rate)),
             20 * nft.fftshift(np.log10(np.abs(nft.fft(input_data)))))
    
    ax1.plot(nft.fftshift(nft.fftfreq(fft_size, d=1/input_rate)),
             20 * np.fft.fftshift(np.log10(np.abs(nft.fft(np.flip(filter_taps), n=fft_size)))))
    
    ax1.plot(nft.fftshift(nft.fftfreq(output_data.size, d=1/output_rate)) + center_freq,
             20 * nft.fftshift(np.log10(np.abs(nft.fft(output_data)))))
    
    ax1.title.set_text('Frequency Domain')
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('Power (dB)')
    ax1.legend(['Input', 'Filter', 'Output (decimated)'])
    if freq_lim is not None:
        ax1.set_xlim(freq_lim[0], freq_lim[1])

    ax2.plot(np.arange(input_data.size)/input_rate, 20 * np.log10(np.abs(input_data)))
    ax2.plot(np.arange(output_data.size)/output_rate, 20 * np.log10(np.abs(output_data)))
    ax2.title.set_text('Time Domain')
    ax2.set_ylabel('Power (a.u.)')
    ax2.set_xlabel('Time (s)')
    ax2.legend(['Input', 'Output (decimated)'])

    plt.show()
    return fig, (ax1, ax2)


def plot_freq_time_comparison(time_data, freq_data, input_rate, title, legend):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize)
    fig.suptitle(title)

    fig.tight_layout(pad=3)

    # Input rate plot
    ax1.plot(nft.fftshift(nft.fftfreq(time_data.size, d=1/input_rate)),
             20 * nft.fftshift(np.log10(np.abs(nft.fft(time_data)))))
    ax1.plot(nft.fftshift(nft.fftfreq(freq_data.size, d=1/input_rate)),
             20 * nft.fftshift(np.log10(np.abs(nft.fft(freq_data)))))
    ax1.title.set_text('Frequency Domain')
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('Power (dB)')
    ax1.legend(legend)

    ax2.plot(np.arange(time_data.size)/input_rate, 20 * np.log10(np.abs(time_data)))
    ax2.plot(np.arange(freq_data.size)/input_rate, 20 * np.log10(np.abs(freq_data)))
    ax2.title.set_text('Time Domain')
    ax2.set_ylabel('Power (a.u.)')
    ax2.set_xlabel('Time (s)')
    ax2.legend(legend)

    plt.show()
    return fig, (ax1, ax2)


def plot_sequence(sequence_data, input_rate, title, freq_lim=None, time_lim=None):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize)
    fig.suptitle(title)

    fig.tight_layout(pad=3)

    # Input rate plot
    ax1.plot(nft.fftshift(nft.fftfreq(sequence_data.size, d=1/input_rate)),
             20 * nft.fftshift(np.log10(np.abs(nft.fft(sequence_data)))))
    ax1.title.set_text('Frequency Domain')
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('Power (dB)')
    if freq_lim is not None:
        ax1.set_xlim(freq_lim[0], freq_lim[1])

    ax2.plot(np.arange(sequence_data.size)/input_rate, np.real(sequence_data))
    ax2.title.set_text('Time Domain')
    ax2.set_ylabel('Power (a.u.)')
    ax2.set_xlabel('Time (s)')
    if time_lim is not None:
        ax2.set_xlim(time_lim[0], time_lim[1])
    
    plt.show()
    return fig, (ax1, ax2)

## The default DecimationScheme
First, we can use the default DecimationScheme object provided by Borealis. 

The default scheme consists of four stages, representing four filtering and downsampling operations. Each stage has an input sampling rate, a downsampling factor, an output sample rate, and a list of filter taps. The frequency response of each stage is plotted by the following block. Note that the frequency span is smaller for each successive stage, since the output of the previous stage is downsampled.

In [ ]:
from src.experiment_prototype.experiment_utils import decimation_scheme as decimation

default_scheme = decimation.create_default_scheme()

print(default_scheme)


for i in range(len(default_scheme.stages)):
    print(np.min(np.abs(default_scheme.stages[i].filter_taps)))
    fig, ax = plt.subplots(1, 1)
    phase_ax = ax.twinx()
    plot_stage_freq_response_on_ax(fig, ax, default_scheme.stages[i], default_scheme.input_rates[i])
    plot_stage_phase_response_on_ax(fig, phase_ax, default_scheme.stages[i], default_scheme.input_rates[i], color='tab:orange')

## Equivalent single-stage default scheme

While the frequency response of each stage above is insightful, the combined filter response is more complicated. The downsampling operation aliases out-of-band signals into the result. Take for example the second filter stage of the default scheme. The sampling rate for this stage is 500 kHz, corresponding to a downsampling factor of 10 from the original 5 MHz sampling rate. The second filter stage has a frequency response defined between -250 to 250 kHz. However, since the input data was downsampled, the frequency response of this filter in the input domain actually has 10 copies: the original copy, between -250 to 250 kHz, plus copies from 250 to 750 kHz, 750 to 1250 kHz, etc., and the same for negative frequencies, plus one final copy from -2.5 MHz to -2.25 MHz and wrapping around at 2.25 to 2.5 MHz. If the input spectrum contains a strong interfering signal in the passband of one of these copies, there can be significant leakage into the filter output. These copies are suppressed by the first filter stage, so a large sidelobe suppression is desired in the first stage.

In [ ]:
def equivalent_single_stage_scheme(scheme: decimation.DecimationScheme):
    """
    Calculates an equivalent single-stage DecimationScheme.
    """
    if len(scheme.stages) == 1:
        return scheme

    taps = [stage.filter_taps for stage in scheme.stages]
    dm_rate_so_far = 1
    for i, f in enumerate(taps):
        filter = np.zeros(len(f) * dm_rate_so_far - (dm_rate_so_far - 1), dtype=np.float32)
        filter[::dm_rate_so_far] = f
        if i == 0:
            total_filter = filter
        else:
            total_filter = np.convolve(total_filter, filter)
        dm_rate_so_far *= scheme.dm_rates[i]
    filter_taps = total_filter.tolist()
    single_stage = decimation.DecimationStage(0, scheme.rxrate, dm_rate_so_far, filter_taps)
    equivalent_scheme = decimation.DecimationScheme(scheme.rxrate, scheme.rxrate / dm_rate_so_far, stages=[single_stage])

    return equivalent_scheme

Let's first take a look at the copies of the second filter stage viewed at the input sampling rate. We can mock this by creating a first filter stage that is an all-pass filter.

In [ ]:
def mock_scheme():
    default = decimation.create_default_scheme()
    first_stage = decimation.DecimationStage(0, default.rxrate, 10, [1.0])
    second_stage = default.stages[1]

    return decimation.DecimationScheme(default.rxrate, default.rxrate / (10 * default.dm_rates[1]), stages=[first_stage, second_stage])

second_stage_scheme = equivalent_single_stage_scheme(mock_scheme())
print(second_stage_scheme)

fig, ax = plt.subplots(1, 1)
phase_ax = ax.twinx()
plot_stage_freq_response_on_ax(fig, ax, second_stage_scheme.stages[0], second_stage_scheme.input_rates[0])
plot_stage_phase_response_on_ax(fig, phase_ax, second_stage_scheme.stages[0], second_stage_scheme.input_rates[0], color='tab:orange')

As we can see, there are 10 peaks in the above figure, one for each copy of the filter. Since the first filter stage was an all-pass filter, there is no suppression of the unwanted copies.

With this in mind, let's then look at the first two stages of the default scheme, which should suppress the copies significantly due to the first filter stage.

In [ ]:
def first_two_stage_scheme():
    default = decimation.create_default_scheme()
    first_stage = default.stages[0]
    second_stage = default.stages[1]

    return decimation.DecimationScheme(default.rxrate, default.rxrate / (default.dm_rates[0] * default.dm_rates[1]), stages=[first_stage, second_stage])

first_two_stages = equivalent_single_stage_scheme(first_two_stage_scheme())
print(first_two_stages)

fig, ax = plt.subplots(1, 1)
phase_ax = ax.twinx()
plot_stage_freq_response_on_ax(fig, ax, first_two_stages.stages[0], first_two_stages.input_rates[0])
plot_stage_phase_response_on_ax(fig, phase_ax, first_two_stages.stages[0], first_two_stages.input_rates[0], color='tab:orange')

The copies have been suppressed by the first filter stage. This is expected, since the first filter stage has 150 dB of sidelobe suppression.

## Equivalent default scheme (all stages)
With downsampling now fully considered, we can evaluate the combined frequency response of the default scheme.

In [ ]:
equivalent_default = equivalent_single_stage_scheme(default_scheme)

print(equivalent_default)

fig, ax = plt.subplots(1, 1)
phase_ax = ax.twinx()
plot_stage_freq_response_on_ax(fig, ax, equivalent_default.stages[0], equivalent_default.input_rates[0])
plot_stage_phase_response_on_ax(fig, phase_ax, equivalent_default.stages[0], equivalent_default.input_rates[0], color='tab:orange')

This frequency response is sharply peaked, with linear phase inside the passband, 150 dB of sidelobe suppression, and an efficient multi-stage implementation. Filter design is about tradeoffs - this scheme works well, but it is worth investigating these tradeoffs to see if a "better" filter can be found.

## Creating new filters
There are functions in the `decimation` module which are helpful for creating new filters. For simplicity, we'll first examine the functions one by one. For more information about the different windowing functions used by `decimation` functions, consult https://docs.scipy.org/doc/scipy/reference/signal.windows.html

In [ ]:
def create_scheme_from_taps_lists(taps_list):
    """ Helper function to make DecimationScheme objects"""
    dm_rate_so_far = 1
    stages = []
    for i, taps in enumerate(taps_list):
        stages.append(decimation.DecimationStage(i, sample_rate, 1, taps))
    return decimation.DecimationScheme(sample_rate, sample_rate, stages=stages)

sample_rate = 5e6  # 5 MHz
cutoff_hz = 100e3  # 100 kHz
transition_width = 100e3   # 100 kHz
ripple_db = 150   # stopband suppression
num_taps = 1500

firwin_by_atten = decimation.create_firwin_filter_by_attenuation(sample_rate, transition_width, cutoff_hz, ripple_db, window_type='kaiser')
firwin_by_num_taps = decimation.create_firwin_filter_by_num_taps(sample_rate, cutoff_hz, num_taps, window_type=('kaiser', 8.0))
remez_filter = decimation.create_remez_filter(sample_rate, cutoff_hz, transition_width)

# Print out the length of each filter
print(f'firwin_by_atten length: {len(firwin_by_atten)}')
print(f'firwin_by_num_taps length: {len(firwin_by_num_taps)}')
print(f'remez_filter length: {len(remez_filter)}')

firwin_by_atten_scheme = create_scheme_from_taps_lists([firwin_by_atten.tolist()])
firwin_by_num_taps_scheme = create_scheme_from_taps_lists([firwin_by_num_taps.tolist()])
remez_scheme = create_scheme_from_taps_lists([remez_filter.tolist()])

titles = ["Firwin by Attenuation", "Firwin by # taps", "Firwin by Remez Algorithm"]
for scheme, title in zip([firwin_by_atten_scheme, firwin_by_num_taps_scheme, remez_scheme], titles):
    fig, ax = plt.subplots(1, 1)
    phase_ax = ax.twinx()
    fig.suptitle(title)
    plot_stage_freq_response_on_ax(fig, ax, scheme.stages[0], scheme.input_rates[0])
    plot_stage_phase_response_on_ax(fig, phase_ax, scheme.stages[0], scheme.input_rates[0], color='tab:orange')

It is apparent that the remez filter does not perform as well as the two Kaiser windows. There is always a tradeoff, however; the remez filter is much shorter, at only 150 taps. Generally, the number of filter taps scales inversely to the transition width between the passband and stopband. The smaller the transition width, the more taps your filter will have. 

The other consideration with filter design is calculation speed. For a single-stage filter, the speed will increase with the number of filter taps. The downsampling factor of the filter is also very important, as it sets how many multiplications can be skipped. A high downsampling factor will greatly increase the speed of a filter stage. For example, if we make a new two-stage filtering scheme that has the same input and output sampling rates as the default scheme, but downsamples by 30 and 50 after each stage, the total number of calculations required is smaller. Additionally, the amount of memory required to store the results of the filter operation is reduced, since the intermediate results are at a lower sampling rate compared to the default scheme.

In [ ]:
def two_stage_filter():
    """
    Two-stage kaiser window scheme.

    Works well with the following parameters:
    sample_rate = 5e6
    dm_rate = [30, 50]
    transition_width = [150e3, 25e3]
    cutoff_hz = [10e3, 5e3]
    ripple_db = [115, 50]
    """
    sample_rate = 5e6                    # 5 MHz
    dm_rate = [30, 50]                   # downsampling rates after filters
    transition_width = [150e3, 30e3]     # transition from passband to stopband
    cutoff_hz = [10e3, 5e3]              # bandwidth for output of filter
    ripple_db = [115, 50]                # dB between passband and stopband
    scaling_factors = [1000., 10000.]    # multiplicative factors for each filter stage
    
    dm_rate_so_far = 1
    stages = []
    for i in range(2):
        rate = sample_rate / dm_rate_so_far
        taps = scaling_factors[i] * decimation.create_firwin_filter_by_attenuation(
            rate, transition_width[i], cutoff_hz[i], ripple_db[i]
        )
        stages.append(decimation.DecimationStage(i, rate, dm_rate[i], taps.tolist()))
        dm_rate_so_far *= dm_rate[i]

    scheme = decimation.DecimationScheme(sample_rate, sample_rate / dm_rate_so_far, stages=stages)

    return scheme

new_filter = two_stage_filter()
print('************** Two-stage filter **************')
print(new_filter)

equivalent_new_filter = equivalent_single_stage_scheme(new_filter)
print('\n************** Equivalent single-stage filter **************')
print(equivalent_new_filter)

fig, ax = plt.subplots(1, 1)
phase_ax = ax.twinx()
plot_stage_freq_response_on_ax(fig, ax, equivalent_new_filter.stages[0], equivalent_new_filter.input_rates[0])
plot_stage_phase_response_on_ax(fig, phase_ax, equivalent_new_filter.stages[0], equivalent_new_filter.input_rates[0], color='tab:orange')

We can see some copies of the second filter stage in near the central passband. The first filter stage only suppresses side lobes by 115 dB, so the closest two copies on either side are above the noise floor. Is this acceptable? It is up to the filter designer and the requirements of the filter.

# Testing on real data
Included in this directory is a single sequence of data taken from a `rawrf` file. The radar was transmitting at 10.5 MHz at this time, with a center frequency of 12.0 MHz. First, we load in the data, then plot the time and frequency domain representations to gain some insight on the spectrum.

In [ ]:
example_data = np.load("example_sequence.npz")
sequence_samples_cpu = example_data['sqn_data']
# The sequence is prepended with data before the start of the first pulse, assuming the default DecimationScheme was to be used. 
default_extra_samples = 0
taps = [stage.filter_taps for stage in default_scheme.stages]
for dm, taps in zip(reversed(default_scheme.dm_rates), reversed(taps)):
    default_extra_samples = (default_extra_samples * dm) + len(taps) // 2
print('Extra samples: ', default_extra_samples)

_ = plot_sequence(sequence_samples_cpu, sample_rate, 'Sequence Data')
# plot_sequence(sequence_samples_cpu, sample_rate, 'Sequence Data', freq_lim=(-1.75e6, -1.25e6))

From the frequency domain representation, it is clear that there are a lot of significant signals within the band. The transmitted signal is at an offset frequency of -1.5 MHz, and is far from the strongest signal. This is a good test of how well the filtering schemes can remove unwanted interference from other sources in the band.

## Default filtering scheme
First, let's test out the default filtering scheme that Borealis uses.

In [ ]:
def mock_dsp(input_samples, mixing_freqs, scheme: decimation.DecimationScheme):
    try:
        import cupy as cp
        cupy_available = True
    except ImportError:
        cupy_available = False

    filter_taps = [np.array(stage.filter_taps, dtype=np.complex64) for stage in scheme.stages]

    # Instantiate the DSP object
    processor = DSP(scheme.rxrate, filter_taps, mixing_freqs, scheme.dm_rates, use_shared_mem=False)

    # Here we account for the edge effects of the filter, to ensure that the first sample in the filtered data aligns
    # with the first transmitted pulse. This is done automatically when Borealis runs.
    extra_samples = 0
    for dm, stage_taps in zip(reversed(scheme.dm_rates), reversed(filter_taps)):
        extra_samples = (extra_samples * dm) + len(stage_taps) // 2
    sample_diff = default_extra_samples - extra_samples
    if sample_diff >= 0:
        input_samples = input_samples[:, sample_diff:]
    else: 
        print((input_samples.shape[0], input_samples.shape[1] + np.abs(sample_diff)))
        padded_input = np.zeros((input_samples.shape[0], input_samples.shape[1] + np.abs(sample_diff)), dtype=input_samples.dtype)
        padded_input[:, -sample_diff:] = input_samples
        input_samples = padded_input
        
    # Move samples to GPU if cupy is being used.
    if cupy_available:
        input_samples = cp.asarray(input_samples)

    # Now actually apply the filter to the data
    processor.apply_filters(input_samples)

    filter_outputs = processor.filter_outputs
    mixed_filters = processor.filters
    
    # Move the results to the CPU (if GPU is used, this is needed)
    if cupy_available:
        cpu_outputs = []
        mixed_filters_cpu = []
        for i in range(len(filter_outputs)):
            cpu_outputs.append(cp.asnumpy(filter_outputs[i]))
            mixed_filters_cpu.append(cp.asnumpy(mixed_filters[i]))
        filter_outputs = cpu_outputs
        mixed_filters = mixed_filters_cpu

    del processor
        
    return filter_outputs, mixed_filters

def plot_dsp_process(scheme, input_samples, output_samples, filter_taps, mixing_freqs, zoom_in=False, freq_lim=None):
    for i in range(len(scheme.stages)):
        antenna_output = output_samples[i][0, 0, :]
        filter = filter_taps[i]
        
        if zoom_in:
            freq_lim = np.array([-scheme.stages[i].output_rate, scheme.stages[i].output_rate])
        else:
            freq_lim = freq_lim
            
        if i == 0:
            if zoom_in:
                freq_lim -= mixing_freqs[0]
            title = f'Filtered Rawrf Samples - Stage {i}'
            plot_filter_stage(input_samples, antenna_output, filter[0, :],
                              scheme.stages[i].input_rate, scheme.stages[i].output_rate,
                              title, center_freq=mixing_freqs[0], freq_lim=freq_lim)
        else:
            antenna_input = output_samples[i-1][0, 0, :]
            title = f'Filtered Rawrf Samples - Stage {i}'
            plot_filter_stage(antenna_input, antenna_output, filter[0, :],
                              scheme.stages[i].input_rate, scheme.stages[i].output_rate,
                              title, freq_lim=freq_lim)

mixing_freq = [-1.5e6]
sequence_samples = sequence_samples_cpu[np.newaxis, :]
default_outputs, mixed_filters = mock_dsp(sequence_samples, mixing_freq, default_scheme)

# plot the results
plot_dsp_process(default_scheme, sequence_samples[0], default_outputs, mixed_filters, mixing_freq)
# plot_dsp_process(default_scheme, sequence_samples[0], default_outputs, mixed_filters, mixing_freq, zoom_in=True)

## New filter usage on real data
Now, we can compare with the performance of the new filter on the real data.

In [ ]:
new_outputs, new_mixed_filters = mock_dsp(sequence_samples, mixing_freq, new_filter)

# plot the results
plot_dsp_process(new_filter, sequence_samples[0], new_outputs, new_mixed_filters, mixing_freq)

## Direct comparison of filter outputs
Now, we plot the output of both the default filter scheme and the new filter scheme, to compare how the resulting antennas_iq data looks.

In [ ]:
final_data_1 = default_outputs[-1]
final_data_2 = new_outputs[-1]

antenna_data_1 = final_data_1[0, 0, :]
antenna_data_2 = final_data_2[0, 0, :]
legend = ['Default Scheme', 'New Scheme']
title = f'Filter Scheme Comparison'
_ = plot_freq_time_comparison(antenna_data_1, antenna_data_2, default_scheme.output_rates[-1], title, legend)

## Benchmark filter performance
It is important to check the runtime of a filter on realistic data when comparing filters. If using a GPU, the memory usage may also be important given the data volume Borealis produces.

**NOTE:** This benchmark is only using data from a single antenna, whereas Borealis typically operates on data streams from 20 antennas. Feel free to change the `num_antennas` field below to get a more realistic benchmark, or to add more entries to `mixing_freqs` to benchmark performance in `CONCURRENT` slice experiments.

In [ ]:
def benchmark_dsp(input_samples, mixing_freqs, scheme: decimation.DecimationScheme):
    import time
    try:
        import cupy as cp
        cupy_available = True
    except ImportError:
        cupy_available = False

    filter_taps = [np.array(stage.filter_taps, dtype=np.complex64) for stage in scheme.stages]

    # Instantiate the DSP object
    processor = DSP(scheme.rxrate, filter_taps, mixing_freqs, scheme.dm_rates, use_shared_mem=False)

    # Here we account for the edge effects of the filter, to ensure that the first sample in the filtered data aligns
    # with the first transmitted pulse. This is done automatically when Borealis runs.
    extra_samples = 0
    for dm, stage_taps in zip(reversed(scheme.dm_rates), reversed(filter_taps)):
        extra_samples = (extra_samples * dm) + len(stage_taps) // 2
    sample_diff = default_extra_samples - extra_samples
    if sample_diff >= 0:
        input_samples = input_samples[:, sample_diff:]
    else: 
        print((input_samples.shape[0], input_samples.shape[1] + np.abs(sample_diff)))
        padded_input = np.zeros((input_samples.shape[0], input_samples.shape[1] + np.abs(sample_diff)), dtype=input_samples.dtype)
        padded_input[:, -sample_diff:] = input_samples
        input_samples = padded_input
        
    # Move samples to GPU if cupy is being used.
    if cupy_available:
        input_samples = cp.asarray(input_samples)
        mempool = cp.get_default_memory_pool()
        start_bytes = mempool.used_bytes()
    
    # Now actually apply the filter to the data. This we do in a loop, and remove the first few runs for warming up.
    sum = 0
    for i in range(1100):
        if cupy_available:
            cp.cuda.Stream.null.synchronize()
        start = time.perf_counter_ns()
        processor.apply_filters(input_samples)
        if cupy_available:
            cp.cuda.Stream.null.synchronize()
        end = time.perf_counter_ns()
        if i > 100:
           sum += end - start

    if cupy_available:
        end_bytes = mempool.used_bytes()
        processor.filter_outputs = None
        print(f'GPU memory used: {float(end_bytes - start_bytes) / 1e6:.2f} MB')
    print(f'Average runtime: {(float(sum) / 1000) * 1e-6:.3f} ms')

In [ ]:
num_antennas = 2
mixing_freqs = [-1.5e6]

bench_samples = np.repeat(sequence_samples, num_antennas, axis=0)

print('********* Default Filter ***********')
benchmark_dsp(bench_samples, mixing_freqs, default_scheme)
print('********* New Filter ***********')
benchmark_dsp(bench_samples, mixing_freqs, new_filter)

# Filter Testbed
Now, feel free to create your own filters here and compare their performance, both theoretically and on the real data introduced above!

In [ ]:
def single_stage_filter():
    sample_rate = 5e6                    # 5 MHz
    dm_rate = [1500]                   # downsampling rates after filters
    transition_width = [30e3]     # transition from passband to stopband
    cutoff_hz = [5e3]              # bandwidth for output of filter
    ripple_db = [115]                # dB between passband and stopband
    scaling_factors = [1e7]    # multiplicative factors for each filter stage
    
    dm_rate_so_far = 1
    stages = []
    for i in range(1):
        rate = sample_rate / dm_rate_so_far
        taps = scaling_factors[i] * decimation.create_firwin_filter_by_attenuation(
            rate, transition_width[i], cutoff_hz[i], ripple_db[i]
        )
        stages.append(decimation.DecimationStage(i, rate, dm_rate[i], taps.tolist()))
        dm_rate_so_far *= dm_rate[i]

    scheme = decimation.DecimationScheme(sample_rate, sample_rate / dm_rate_so_far, stages=stages)

    return scheme

new_filter = single_stage_filter()
print('************** Single-stage filter **************')
print(new_filter)

fig, ax = plt.subplots(1, 1)
phase_ax = ax.twinx()
plot_stage_freq_response_on_ax(fig, ax, new_filter.stages[0], new_filter.input_rates[0])
plot_stage_phase_response_on_ax(fig, phase_ax, new_filter.stages[0], new_filter.input_rates[0], color='tab:orange')

filter_taps = [np.array(stage.filter_taps, dtype=np.complex64) for stage in new_filter.stages]
new_outputs, new_mixed_filters = mock_dsp(sequence_samples, mixing_freq, new_filter)

# plot the results
plot_dsp_process(new_filter, sequence_samples[0], new_outputs, new_mixed_filters, mixing_freq, freq_lim=(-1.55e6, -1.45e6))

final_data_3 = new_outputs[-1]
antenna_data_3 = final_data_3[0, 0, :]
legend = ['Default Scheme', 'New Scheme']
title = f'Filter Scheme Comparison'
_ = plot_freq_time_comparison(antenna_data_1, antenna_data_3, default_scheme.output_rates[-1], title, legend)
legend = ['New Two-stage Scheme', 'New Single-stage Scheme']
_ = plot_freq_time_comparison(antenna_data_2, antenna_data_3, default_scheme.output_rates[-1], title, legend)

# Bandwidth of transmitted pulses
The actual pulse train sent out by the radar has a bandwidth determined by the envelope of each individual pulse. A linear ramp up/down is used for each pulse. Let's investigate the spectral properties of the pulse!

In [ ]:
from src.utils.signals import get_samples

pulse_len = 0.0003
ramp_time = 0.00001
max_amplitude = 1.0
default_samples = get_samples(sample_rate, mixing_freq[0], pulse_len, ramp_time, max_amplitude)
default_samples.shape
padded_samples = np.zeros(16384, dtype=np.complex64)
padded_samples[:default_samples.shape[0]] = default_samples

In [ ]:
_ = plot_sequence(padded_samples, sample_rate, 'Basic Pulse', freq_lim=None, time_lim=(0.0, 0.0003))

## Different pulse ramps
Linear ramps (or equivalently, a triangular window) are rather poor in the spectral domain. Better ramps exist; see the `scipy.signal.windows` module for a list of some common functions and their properties.
For comparison's sake, several windows are shown below. These include the uniform (boxcar), triangular, Gaussian, Blackman-Harris, and Bohman windows. The last three have significantly better spectral properties than the uniform and triangular window.

In [ ]:
def compare_pulse_ramps():
    fig, axes = plt.subplots(3, 1, figsize=(10, 10))

    def plot_spectrum(ax, data, fs, label=None):
        ax.plot(nft.fftshift(nft.fftfreq(data.size, d=1/fs)),
             20 * nft.fftshift(np.log10(np.abs(nft.fft(data)))), label=label)
        
    rate = float(sample_rate)
    sampling_freq = 2 * np.pi * mixing_freq[0] / rate
    num_samples = round(rate * pulse_len)
    ramp_len = 101
    half_ramp = ramp_len // 2 + 1
    rads = sampling_freq * np.arange(num_samples)
    wave_form = np.exp(rads * 1j)

    # Generate some test ramps
    import scipy.signal
    gaussian = scipy.signal.windows.gaussian(ramp_len, std=15)
    bohman = scipy.signal.windows.bohman(ramp_len)
    blackman_harris = scipy.signal.windows.blackmanharris(ramp_len)
    triangle = scipy.signal.windows.triang(ramp_len)
    uniform = np.ones(ramp_len)
    
    # Add/remove a tuple from this list to remove that ramp from the plot.
    ramps = [
        (uniform, "Uniform"),
        (triangle, "Triangle"), 
        (gaussian, "Gaussian"), 
        (blackman_harris, "Blackman-Harris"), 
        (bohman, "Bohman"), 
    ]

    # Iterate through the ramps and plot their
    for ramp, label in ramps:
        wave_form = np.exp(rads * 1j)

        amplitude_ramp_up = ramp[:half_ramp]
        amplitude_ramp_down = np.flipud(amplitude_ramp_up)
        
        ramp_up_piece = wave_form[:half_ramp]
        ramp_down_piece = wave_form[num_samples - half_ramp:]
        np.multiply(ramp_up_piece, amplitude_ramp_up, out=ramp_up_piece)
        np.multiply(ramp_down_piece, amplitude_ramp_down, out=ramp_down_piece)
        
        samples = wave_form * max_amplitude
        padded_samples = np.zeros(16384, dtype=np.complex64)
        padded_samples[:samples.shape[0]] = samples
    
        axes[0].plot(ramp[:half_ramp], label=label)
        padded_ramp = np.zeros(16384, dtype=np.complex64)
        padded_ramp[:ramp_len] = ramp
        plot_spectrum(axes[1], padded_ramp, sample_rate, label=label)
        # axes[0].plot(padded_samples[:100], label=label)
        plot_spectrum(axes[2], padded_samples, sample_rate, label=label)
    axes[0].legend()
    plt.show()

compare_pulse_ramps()